# Testing CBOR implementations for map encoding canonicality

Tested:
- [Julia](https://github.com/JuliaIO/CBOR.jl)
- [Rust](https://docs.rs/serde_cbor/0.11.0/serde_cbor/)
- [JS cbor](https://www.npmjs.com/package/cbor)
- [JS borc](https://www.npmjs.com/package/borc)

In [216]:
using CBOR, JSON

In [217]:
data_dict = Dict("a" => 1, "b" => 2, "c" => Dict("b" => 1, "a" => 2))
data_json = json(data_dict)

"{\"c\":{\"b\":1,\"a\":2},\"b\":2,\"a\":1}"

In [218]:
data_json = """{"iss":"BBmEKZciGMonT_G0CmiM4HdfM6o0ktuh3xIFadvc1TVgA0ZJUNIS6go0pX8jwSUorbDfv27T_M9M9wldMFk6t00","iat":1578835560,"sub":"geo:?q=47.1691576,8.514572(Juanitos)&u=30","rating":75,"metadata":{"display_name":"dsafsa","given_name":"sadf","age":23,"gender":"Female","client_uri":"https://mangrove.reviews"}}"""
data_dict = JSON.parse(data_json)

Dict{String,Any} with 5 entries:
  "sub"      => "geo:?q=47.1691576,8.514572(Juanitos)&u=30"
  "iat"      => 1578835560
  "iss"      => "BBmEKZciGMonT_G0CmiM4HdfM6o0ktuh3xIFadvc1TVgA0ZJUNIS6go0pX8jwS…
  "rating"   => 75
  "metadata" => Dict{String,Any}("client_uri"=>"https://mangrove.reviews","gend…

In [219]:
encoded_jl = encode(data_dict)
length(encoded_jl)

280

In [220]:
encoded_js_borc = `node js/cbor.js borc encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_js_borc)

261

In [221]:
encoded_js_cbor = `node js/cbor.js cbor encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_js_cbor)

261

In [222]:
encoded_rs = `./rust/target/debug/rust encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_rs)

261

In [223]:
@assert(encoded_js_borc == encoded_js_cbor)

AssertionError: AssertionError: encoded_js_borc == encoded_js_cbor

In [224]:
@assert(encoded_js_borc == encoded_rs)

In [225]:
@assert(encoded_rs == encoded_jl)

AssertionError: AssertionError: encoded_rs == encoded_jl